Load packages

In [5]:
#LOAD PACKAGES
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import pandas as pd
from shapely.geometry import LineString, Point
import requests
import pandas as pd
import os
import folium

import pyproj
#pyproj.datadir.set_data_dir("/opt/homebrew/share/proj")

Data Cleaning (from Krystal)

In [6]:
data_dir = "Bus_RoutesStopsServices"
bus_routes = pd.read_csv(f"{data_dir}/bus_routes.csv")
bus_services = pd.read_csv(f"{data_dir}/bus_services.csv")
bus_stops = pd.read_csv(f"{data_dir}/bus_stops.csv")

# bus services data: filter for trunk, and direction = 1
trunk = bus_services[bus_services['Category'] == 'TRUNK']
trunk = trunk[trunk['Direction'] == 1]

#bus routes data: filter for direction = 1 and trunk buses
bus_routes = bus_routes[bus_routes['Direction'] == 1]
merged_data = pd.merge(bus_routes, trunk[['ServiceNo']], on='ServiceNo', how='inner')
merged_data

# join to get the bus stop coordinates
final_data = pd.merge(merged_data, bus_stops, on='BusStopCode', how='left')
final_data

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,RoadName,Description,Latitude,Longitude
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300,Tampines Ctrl 1,Tampines Int,1.354076,103.943391
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302,Tampines Ave 5,Opp Our Tampines Hub,1.352962,103.941652
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304,Tampines Ave 5,Blk 147,1.348753,103.942086
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308,Simei Ave,Changi General Hosp,1.340055,103.948381
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309,Simei Ave,Opp Blk 3012,1.337371,103.950673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14821,9B,SBST,1,25,95091,9.5,0741,0817,-,-,-,-,Nicoll Dr,SAF Ferry Ter,1.387214,103.999714
14822,9B,SBST,1,26,95131,9.7,0742,0818,-,-,-,-,Airline Rd,Police Pass Off,1.385252,103.999834
14823,9B,SBST,1,27,95141,10.2,0744,0820,-,-,-,-,Airline Rd,Aft Cargo Bldg D,1.381386,103.998191
14824,9B,SBST,1,28,95061,10.6,0745,0821,-,-,-,-,Airline Rd,Airline Hse,1.377552,103.996539


Train Routes Plotting + Pop Up

In [7]:
import folium
import geopandas as gpd
import pandas as pd

train_stations = gpd.read_file("TrainStation_Jul2024/repaired_shapefile.shp")

mrt_lines = {
    'Thomson-East Coast Line': ['WOODLANDS NORTH MRT STATION','WOODLANDS MRT STATION','WOODLANDS SOUTH MRT STATION', 'SPRINGLEAF MRT STATION','LENTOR MRT STATION','MAYFLOWER MRT STATION','BRIGHT HILL MRT STATION',
                                'UPPER THOMSON MRT STATION','CALDECOTT MRT STATION','STEVENS MRT STATION','NAPIER MRT STATION','ORCHARD BOULEVARD MRT STATION','ORCHARD MRT STATION','GREAT WORLD MRT STATION',
                                'HAVELOCK MRT STATION','OUTRAM PARK MRT STATION','MAXWELL MRT STATION','SHENTON WAY MRT STATION','MARINA BAY MRT STATION','GARDENS BY THE BAY MRT STATION','TANJONG RHU MRT STATION',
                                'KATONG PARK MRT STATION','TANJONG KATONG MRT STATION','MARINE PARADE MRT STATION','MARINE TERRACE MRT STATION','SIGLAP MRT STATION','BAYSHORE MRT STATION'],
    'North-South Line': ['JURONG EAST MRT STATION','BUKIT BATOK MRT STATION','BUKIT GOMBAK MRT STATION','CHOA CHU KANG MRT STATION','YEW TEE MRT STATION','KRANJI MRT STATION','MARSILING MRT STATION','WOODLANDS MRT STATION',
                         'ADMIRALTY MRT STATION','SEMBAWANG MRT STATION','CANBERRRA MRT STATION','YISHUN MRT STATION','KHATIB MRT STATION','YIO CHU KANG MRT STATION','ANG MO KIO MRT STATION','BISHAN MRT STATION',
                         'BRADDELL MRT STATION','TOA PAYOH MRT STATION','NOVENA MRT STATION','NEWTON MRT STATION','ORCHARD MRT STATION','SOMERSET MRT STATION','DHOBY GHAUT MRT STATION','CITY HALL MRT STATION',
                         'RAFFLES PLACE MRT STATION','MARINA BAY MRT STATION','MARINA SOUTH PIER MRT STATION'],
    'Circle Line': ['DHOBY GHAUT MRT STATION', 'BRAS BASAH MRT STATION', 'ESPLANADE MRT STATION', 'PROMENADE MRT STATION','NICOLL HIGHWAY MRT STATION','STADIUM MRT STATION',
                    'MOUNTBATTEN MRT STATION','DAKOTA MRT STATION','PAYA LEBAR MRT STATION','MACPHERSON MRT STATION','TAI SENG MRT STATION','BARTLEY MRT STATION','SERANGOON MRT STATION','LORONG CHUAN MRT STATION',
                    'BISHAN MRT STATION','MARYMOUNT MRT STATION','CALDECOTT MRT STATION','BOTANIC GARDENS MRT STATION','FARRER ROAD MRT STATION','HOLLAND VILLAGE MRT STATION','BUONA VISTA MRT STATION','ONE-NORTH MRT STATION',
                    'KENT RIDGE MRT STATION', 'HAW PAW VILLA MRT STATION', 'PASIR PANJANG MRT STATION', 'LABRADOR PARK MRT STATION', 'TELOK BLANGAH MRT STATION','HABOURFRONT MRT STATION'],
    'Circle Line.': ['PROMENADE MRT STATION','BAYFRONT MRT STATION', 'MARINA BAY MRT STATION'],
    'North-East Line': ['HARBOURFRONT MRT STATION','OUTRAM PARK MRT STATION','CHINATOWN MRT STATION','CLARKE QUAY MRT STATION','DHOBY GHAUT MRT STATION','LITTLE INDIA MRT STATION','FARRER PARK MRT STATION','BOON KENG MRT STATION',
                        'POTONG PASIR MRT STATION','WOODLEIGH MRT STATION','SERANGOON MRT STATION','KOVAN MRT STATION','HOUGANG MRT STATION','BUANGKOK MRT STATION','SENGKANG MRT STATION','PUNGGOL MRT STATION'],
    'Downtown Line': ['BUKIT PANJANG MRT STATION', 'CASHEW MRT STATION', 'HILLVIEW MRT STATION', 'HUME MRT STATION', 'BEAUTY WORLD MRT STATION', 'KING ALBERT PARK MRT STATION', 'SIXTH AVENUE','TAN KAH KEE MRT STATION',
                      'BOTANIC GARDENS MRT STATION', 'STEVENS MRT STATION', 'NEWTON MRT STATION', 'LITTLE INDIA MRT STATION', 'ROCHOR MRT STATION', 'BUGIS MRT STATION', 'PROMENADE MRT STATION', 'BAYFRONT MRT STATION',
                      'DOWNTOWN MRT STATION', 'TELOK AYER MRT STATION', 'CHINATOWN MRT STATION', 'FORT CANNING MRT STATION', ' BENCOOLEN MRT STATION', 'JALAN BESAR MRT STATION', 'BENDEMEER MRT STATION', 'GEYLANG  BAHRU MRT STATION',
                      'MATTAR MRT STATION', 'MACPHERSON MRT STATION', 'UBI MRT STATION', 'KAKI BUKIT MRT STATION', ' BEDOK NORTH MRT STATION', 'BEDOK RESERVOIR MRT STATION', 'TAMPINES WEST MRT STATION',
                      'TAMPINES MRT STATION', 'TAMPINES EAST MRT STATION', 'UPPER CHANGI MRT STATION', 'EXPO MRT STATION'],
    'East-West Line': ['PASIR RIS MRT STATION', 'TAMPINES MRT STATION', 'SIMEI MRT STATION', 'TANAH MERAH MRT STATION','BEDOK MRT STATION', 'KEMBANGAN MRT STATION', 'EUNOS MRT STATION', 'PAYA LEBAR MRT STATION',
                       'ALJUNIED MRT STATION', 'KALLANG MRT STATION', ' LAVENDER MRT STATION','BUGIS MRT STATION', 'CITY HALL MRT STATION', 'RAFFLES PLACE MRT STATION','TANJONG PAGAR MRT STATION', 'OUTRAM PARK MRT STATION',
                       'TIONG BAHRU MRT STATION', 'REDHILL MRT STATION', 'QUEENSTOWN MRT STATION', 'COMMONWEALTH MRT STATION','BUONA VISTA MRT STATION', 'DOVER MRT STATION', 'CLEMENTI MRT STATION', 'JURONG EAST MRT STATION',
                       'CHINESE GARDEN MRT STATION', 'LAKESIDE MRT STATION', 'BOON LAY MRT STATION', 'PIONEER MRT STATION','JOO KON MRT STATION',' GUL CIRCLE MRT STATION', 'TUAS CRESCENT MRT STATION', 
                       'TUAS WEST ROAD MRT STATION', 'TUAS LINK MRT STATION'],
    'East-West Line.':['TANAH MERAH MRT STATION','EXPO MRT STATION', 'CHANGI AIRPORT MRT STATION']
}
line_colors = {
    'North-South Line': 'red',
    'East-West Line': 'green',
    'East-West Line.': 'green',
    'Circle Line': 'darkorange',
    'Circle Line.': 'darkorange',
    #'Circle Line.': 'black', # can use this because orange hard to see
    #'Circle Line': 'black',
    'North-East Line': 'purple',
    'Downtown Line': 'blue',
    'Thomson-East Coast Line': 'brown'
}

mrt_line_data = []
for line, stations in mrt_lines.items():
    for index, station in enumerate(stations):
        mrt_line_data.append((station, line, index+1))
mrt_line_df = pd.DataFrame(mrt_line_data, columns=['STN_NAM_DE', 'MRT_LINE', 'STN_SEQUENCE'])

# Merge this DataFrame with your original train_stations DataFrame on the STN_NAM_DE column
train_stations_merged = train_stations.merge(mrt_line_df, on='STN_NAM_DE', how='left')
merged_stations_unique = train_stations_merged.drop_duplicates(subset=['STN_NAM_DE', 'MRT_LINE'])
merged_stations_unique = merged_stations_unique.to_crs(epsg=4326)
grouped_train_lines = merged_stations_unique.groupby(['MRT_LINE']).apply(lambda x: x[['STN_NAM_DE', 'STN_SEQUENCE','geometry']])
grouped_train_lines = grouped_train_lines.sort_values(['MRT_LINE','STN_SEQUENCE'])
grouped_train_lines = grouped_train_lines.to_crs(epsg=4326)

singapore = folium.Map(location=(1.359394, 103.814301), zoom_start=12)
folium.GeoJson(grouped_train_lines, name="Singapore Map").add_to(singapore)

#try here
grouped_train_lines = grouped_train_lines.groupby(level='MRT_LINE') 
for MRT_LINE, group in grouped_train_lines:
    group_sorted = group.sort_values('STN_SEQUENCE')
    coordinates = group_sorted['geometry'].centroid.apply(lambda geom: (geom.y, geom.x)).tolist()
    train_route_line = LineString(coordinates) #plot the line 
    line_color = line_colors.get(MRT_LINE, 'black')

    folium.PolyLine(     #plot train lines in the map
        locations=coordinates,  
        weight=2,          
        color=line_color,     
        opacity=0.7,      
        popup=f"{MRT_LINE}"  # label with MRT_LINE
    ).add_to(singapore)

# Plot MRT stations with popups:

for idx, row in merged_stations_unique.iterrows():
    station_location = row['geometry'].representative_point()
    station_name = row['STN_NAM_DE']
    lat = station_location.y 
    lon = station_location.x 

    # Create a popup with station name and coordinates
    popup_html = f"""
    <div style="font-size: 12px;">
        <strong>{station_name}</strong><br>
        Lat: {lat:.6f}, Lon: {lon:.6f}
    </div>
    """

    folium.Marker(
        location=[lat, lon],  
        popup=folium.Popup(popup_html, max_width=80),  
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(singapore)

singapore

/Users/krystal/.virtualenvs/r-tensorflow/lib/python3.10/site-packages/pyogrio/raw.py:196: RuntimeWarning: TrainStation_Jul2024/repaired_shapefile.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_53901/510749982.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_train_lines = merged_stations_unique.groupby(['MRT_LINE']).apply(lambda x: x[['STN_NAM_DE', 'STN_SEQUENCE','geometry']])
/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_53901/510749982.py:77: UserWarning: Geometry is in a geographic CRS. Resul

Combined Routes Plot (One bus service + MRT map)

In [4]:
singapore = folium.Map(location=(1.359394, 103.814301), zoom_start=12)
# grp data by service number and direction
#busroutes = final_data

#to see indiv bus routes
busroutes = final_data[final_data['ServiceNo'].isin(['163'])]

grouped_bus_routes = busroutes.groupby(['ServiceNo', 'Direction'])

for (service_no, direction), group in grouped_bus_routes:
    
    group_sorted = group.sort_values('StopSequence') #ensure busstop for each bus is in correct sequence
    coordinates = list(zip(group_sorted['Latitude'], group_sorted['Longitude'])) #get the coordinates
    bus_route_line = LineString(coordinates) #plot the line 
    
    #plot bus routes in the map
    folium.PolyLine(
        locations=coordinates,  
        weight=2,          
        color='blue',     
        opacity=0.7,      
        popup=f"Service {service_no}"  # label with bus service number
    ).add_to(singapore)

#add the bus stop points in the map too
for index, row in busroutes.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        popup=f"Bus Stop: {row['BusStopCode']} (Service {row['ServiceNo']})"
    ).add_to(singapore)

train_stations = gpd.read_file("TrainStation_Jul2024/repaired_shapefile.shp")
mrt_line_data = []
for line, stations in mrt_lines.items():
    for index, station in enumerate(stations):
        mrt_line_data.append((station, line, index+1))
mrt_line_df = pd.DataFrame(mrt_line_data, columns=['STN_NAM_DE', 'MRT_LINE', 'STN_SEQUENCE'])

# Merge this DataFrame with your original train_stations DataFrame on the STN_NAM_DE column
train_stations_merged = train_stations.merge(mrt_line_df, on='STN_NAM_DE', how='left')
merged_stations_unique = train_stations_merged.drop_duplicates(subset=['STN_NAM_DE', 'MRT_LINE'])
merged_stations_unique = merged_stations_unique.to_crs(epsg=4326)
grouped_train_lines = merged_stations_unique.groupby(['MRT_LINE']).apply(lambda x: x[['STN_NAM_DE', 'STN_SEQUENCE','geometry']])
grouped_train_lines = grouped_train_lines.sort_values(['MRT_LINE','STN_SEQUENCE'])
grouped_train_lines = grouped_train_lines.to_crs(epsg=4326)

#singapore = folium.Map(location=(1.359394, 103.814301), zoom_start=12)
folium.GeoJson(grouped_train_lines, name="Singapore Map").add_to(singapore)

#try here
grouped_train_lines = grouped_train_lines.groupby(level='MRT_LINE') 
for MRT_LINE, group in grouped_train_lines:
    group_sorted = group.sort_values('STN_SEQUENCE')
    coordinates = group_sorted['geometry'].centroid.apply(lambda geom: (geom.y, geom.x)).tolist()
    train_route_line = LineString(coordinates) #plot the line 
    line_color = line_colors.get(MRT_LINE, 'black')

    folium.PolyLine(     #plot train lines in the map
        locations=coordinates,  
        weight=2,          
        color=line_color,     
        opacity=0.7,      
        popup=f"{MRT_LINE}"  # label with MRT_LINE
    ).add_to(singapore)

# Plot MRT stations with popups:

for idx, row in merged_stations_unique.iterrows():
    station_location = row['geometry'].representative_point()
    station_name = row['STN_NAM_DE']
    lat = station_location.y 
    lon = station_location.x 

    # # IF YOU WANT TO DISPLAY THE POP UPS FOR EACH MRT STATION, UNCOMMENT THE SECTION BELOW
    # popup_html = f"""
    # <div style="font-size: 12px;">
    #     <strong>{station_name}</strong><br>
    #     Lat: {lat:.6f}, Lon: {lon:.6f}
    # </div>
    # """

    # folium.Marker(
    #     location=[lat, lon],  
    #     popup=folium.Popup(popup_html, max_width=80),  
    #     icon=folium.Icon(color='blue', icon='info-sign')
    # ).add_to(singapore)

singapore

/Users/krystal/.virtualenvs/r-tensorflow/lib/python3.10/site-packages/pyogrio/raw.py:196: RuntimeWarning: TrainStation_Jul2024/repaired_shapefile.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_53901/1581913875.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_train_lines = merged_stations_unique.groupby(['MRT_LINE']).apply(lambda x: x[['STN_NAM_DE', 'STN_SEQUENCE','geometry']])
/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_53901/1581913875.py:58: UserWarning: Geometry is in a geographic CRS. Res